In [ ]:
# Additional exploration

In [ ]:
import json
import numpy as np

from task import Task
from utils import plot_sample, set_types, process_task

In [ ]:
train_ch_path = '../input/arc-prize-2024/arc-agi_training_challenges.json'
train_sol_path = '../input/arc-prize-2024/arc-agi_training_solutions.json'
eval_ch_path = '../input/arc-prize-2024/arc-agi_evaluation_challenges.json'
eval_sol_path = '../input/arc-prize-2024/arc-agi_evaluation_solutions.json'
test_path = '../input/arc-prize-2024/arc-agi_test_challenges.json'
sample_path = '../input/arc-prize-2024/sample_submission.json'

with open(train_ch_path, 'r', encoding='utf-8') as file:
    train_tasks = json.load(file)
with open(eval_ch_path, 'r', encoding='utf-8') as file:
    eval_tasks = json.load(file)
with open(test_path, 'r', encoding='utf-8') as file:
    test_tasks = json.load(file)

with open(train_sol_path, 'r', encoding='utf-8') as file:
    truth_t = json.load(file)
with open(eval_sol_path, 'r', encoding='utf-8') as file:
    truth_e = json.load(file)
truth = truth_t | truth_e

print(len(train_tasks), len(eval_tasks), len(test_tasks), len(truth))

In [ ]:
# %%script true
# Task testing
name = '08ed6ac7'
process_task(train_tasks[name])
# for train in test_tasks[name]['train']:
#     plot_sample(train['input'], train['output'])
task = Task(name, train_tasks[name])
task.solve(time_limit=10, verbose=False, shared_frontier=True, constrained=True)

In [ ]:
# %%script true
import cProfile
import pstats

pr = cProfile.Profile()
pr.enable()
task.solve(time_limit=10)
pr.disable()

ps = pstats.Stats(pr).sort_stats('tottime')
_ = ps.print_stats(0.02)

In [ ]:
# Task control
the_tasks = test_tasks
# the_tasks = dict(list(test_tasks.items())[80:99])

In [ ]:
def check_sol(name, sol):
    process_task(the_tasks[name])
    Task(name, the_tasks[name]).check_sol(sol, 'all', True)

the_sol = [{'abstraction': 'nbccg',
            'filters': ['filter_by_nothing'],
            'filter_params': [{}],
            'transformation': ['flip_rotate'],
            'transformation_params': [{'flip_rotation': ('', 90)}]}]

check_sol('4347f46a', the_sol)

In [ ]:
def plot_diffs(tasks):
    set_types(tasks)
    index = 0
    for name, task in tasks.items():
        index += 1
        if index not in range(300, 350) or task['type'] != 1:
            continue
        process_task(task)
        if task['re']:
            print(index, name)
            for i in range(len(task['test'])):
                diff = np.not_equal(task['test'][i]['input'], truth[name][i])
                plot_sample(task['test'][i]['input'], truth[name][i], diff)
            for i in range(len(task['train'])):
                diff = np.not_equal(task['train'][i]['input'], task['train'][i]['output'])
                plot_sample(task['train'][i]['input'], task['train'][i]['output'], diff)

# plot_diffs(the_tasks)

In [ ]:
def explore_score(name, abstraction='nbccg'):
    process_task(the_tasks[name])
    score = Task(name, the_tasks[name]).get_search_score()
    print(score, '-', sum(score.values()))
    random_index = np.random.randint(0, score[abstraction])
    Task(name, the_tasks[name]).get_search_score(abstraction, random_index)

explore_score('1bfc4729')

In [ ]:
def get_search_scores(tasks):
    set_types(tasks)
    index = 0
    for name, task in tasks.items():
        index += 1
        if task['type'] != 1:
            continue
        process_task(task)
        if task['re']:
            score = Task(name, task).get_search_score()
            print('#', index, name, score, '-', sum(score.values()))

# get_search_scores(the_tasks)

In [ ]:
# t100 stable

# 4 025d127b {'nag': 7, 'nbccg': 5915, 'mcccg': 7623, 'nbvccg': 3298, 'nbhccg': 4192} - 21035
# 5 045e512c {'nag': 7, 'nbccg': 22890, 'mcccg': 19412, 'nbvccg': 7566, 'nbhccg': 8563} - 58438
# 7 05269061 {'nag': 7, 'nbccg': 8816, 'mcccg': 2463, 'nbvccg': 4568, 'nbhccg': 4568} - 20422
# 9 06df4c85 {'nag': 7, 'nbccg': 9422, 'mcccg': 1715, 'nbvccg': 10509, 'nbhccg': 10587} - 32240
# 12 0962bcdd {'nag': 7, 'nbccg': 969, 'mcccg': 227, 'nbvccg': 495, 'nbhccg': 495} - 2193
# 13 0a938d79 {'nag': 7, 'nbccg': 2684, 'mcccg': 1732, 'nbvccg': 999, 'nbhccg': 999} - 6421
# 17 0dfd9992 {'nag': 7, 'nbccg': 45642, 'mcccg': 1393, 'nbvccg': 16055, 'nbhccg': 16068} - 79165
# 18 0e206a2e {'nag': 7, 'nbccg': 16911, 'mcccg': 13825, 'nbvccg': 11585, 'nbhccg': 10501} - 52829
# 20 11852cab {'nag': 7, 'nbccg': 12088, 'mcccg': 7743, 'nbvccg': 6269, 'nbhccg': 6269} - 32376
# 25 1a07d186 {'nag': 7, 'nbccg': 10593, 'mcccg': 15745, 'nbvccg': 7742, 'nbhccg': 8841} - 42928
# 28 1bfc4729 {'nag': 7, 'nbccg': 591, 'mcccg': 609, 'nbvccg': 391, 'nbhccg': 391} - 1989
# 32 1e0a9b12 {'nag': 7, 'nbccg': 15402, 'mcccg': 14079, 'nbvccg': 10471, 'nbhccg': 7969} - 47928
# 33 1e32b0e9 {'nag': 7, 'nbccg': 16443, 'mcccg': 9161, 'nbvccg': 4765, 'nbhccg': 5173} - 35549
# 34 1f0c79e5 {'nag': 7, 'nbccg': 4035, 'mcccg': 260, 'nbvccg': 2027, 'nbhccg': 2027} - 8356
# 35 1f642eb9 {'nag': 7, 'nbccg': 3819, 'mcccg': 250, 'nbvccg': 921, 'nbhccg': 860} - 5857
# 37 1f876c06 {'nag': 7, 'nbccg': 11614, 'mcccg': 9195, 'nbvccg': 5918, 'nbhccg': 5931} - 32665
# 40 2204b7a8 {'nag': 7, 'nbccg': 2904, 'mcccg': 114, 'nbvccg': 408, 'nbhccg': 457} - 3890
# 41 22168020 {'nag': 7, 'nbccg': 4258, 'mcccg': 4284, 'nbvccg': 1988, 'nbhccg': 2899} - 13436
# 44 228f6490 {'nag': 7, 'nbccg': 19434, 'mcccg': 41647, 'nbvccg': 13278, 'nbhccg': 14184} - 88550
# 45 22eb0ac0 {'nag': 7, 'nbccg': 9602, 'mcccg': 7950, 'nbvccg': 6487, 'nbhccg': 6487} - 30533
# 51 25d487eb {'nag': 7, 'nbccg': 1877, 'mcccg': 699, 'nbvccg': 4323, 'nbhccg': 4245} - 11151
# 54 264363fd {'nag': 7, 'nbccg': 7671, 'mcccg': 21, 'nbvccg': 1837, 'nbhccg': 1851} - 11387
# 59 29623171 {'nag': 7, 'nbccg': 3765, 'mcccg': 4594, 'nbvccg': 4327, 'nbhccg': 4457} - 17150
# 60 29c11459 {'nag': 7, 'nbccg': 822, 'mcccg': 436, 'nbvccg': 400, 'nbhccg': 400} - 2065
# 61 29ec7d0e {'nag': 7, 'nbccg': 50592, 'mcccg': 2277, 'nbvccg': 28184, 'nbhccg': 31435} - 112495
# 62 2bcee788 {'nag': 7, 'nbccg': 5685, 'mcccg': 1281, 'nbvccg': 4259, 'nbhccg': 4630} - 15862
# 64 2c608aff {'nag': 7, 'nbccg': 3030, 'mcccg': 6391, 'nbvccg': 3364, 'nbhccg': 3351} - 16143
# 69 321b1fc6 {'nag': 7, 'nbccg': 29878, 'mcccg': 3303, 'nbvccg': 9243, 'nbhccg': 10707} - 53138
# 71 3345333e {'nag': 7, 'nbccg': 3369, 'mcccg': 618, 'nbvccg': 6510, 'nbhccg': 4909} - 15413
# 75 363442ee {'nag': 7, 'nbccg': 59022, 'mcccg': 10155, 'nbvccg': 14834, 'nbhccg': 16447} - 100465
# 77 36fdfd69 {'nag': 7, 'nbccg': 6170, 'mcccg': 348, 'nbvccg': 534, 'nbhccg': 578} - 7637
# 80 39e1d7f9 {'nag': 7, 'nbccg': 9303, 'mcccg': 1687, 'nbvccg': 8100, 'nbhccg': 8152} - 27249
# 82 3ac3eb23 {'nag': 7, 'nbccg': 1328, 'mcccg': 1041, 'nbvccg': 661, 'nbhccg': 661} - 3698
# 84 3bd67248 {'nag': 7, 'nbccg': 939, 'mcccg': 1848, 'nbvccg': 342, 'nbhccg': 407} - 3543
# 85 3bdb4ada {'nag': 7, 'nbccg': 1676, 'mcccg': 4080, 'nbvccg': 2512, 'nbhccg': 1777} - 10052
# 86 3befdf3e {'nag': 7, 'nbccg': 4333, 'mcccg': 618, 'nbvccg': 4401, 'nbhccg': 4401} - 13760
# 87 3c9b0459 {'nag': 7, 'nbccg': 3845, 'mcccg': 15, 'nbvccg': 448, 'nbhccg': 534} - 4849
# 89 3e980e27 {'nag': 7, 'nbccg': 36246, 'mcccg': 19813, 'nbvccg': 12601, 'nbhccg': 12614} - 81281
# 90 3eda0437 {'nag': 7, 'nbccg': 20687, 'mcccg': 42603, 'nbvccg': 5971, 'nbhccg': 5711} - 74979
# 92 40853293 {'nag': 7, 'nbccg': 8036, 'mcccg': 10127, 'nbvccg': 5415, 'nbhccg': 5428} - 29013
# 93 4093f84a {'nag': 7, 'nbccg': 5357, 'mcccg': 8109, 'nbvccg': 4349, 'nbhccg': 4414} - 22236
# 97 42a50994 {'nag': 7, 'nbccg': 5706, 'mcccg': 14402, 'nbvccg': 3067, 'nbhccg': 2148} - 25330
# 98 4347f46a {'nag': 7, 'nbccg': 7096, 'mcccg': 15889, 'nbvccg': 10277, 'nbhccg': 10820} - 44089
# 99 444801d8 {'nag': 7, 'nbccg': 5240, 'mcccg': 3984, 'nbvccg': 6442, 'nbhccg': 5170} - 20843